In [1]:
import pandas as pd
import json
import re
from string import punctuation

ModuleNotFoundError: No module named 'pandas'

# pororo 라벨링, 1차 단어사전 만들기

In [ ]:
%%writefile naver_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('Naver종토방')[1:]:
  NAVER = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('Naver종토방/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa),
      NAVER = NAVER.append(j, ignore_index = True)
      NAVER.to_pickle(f"./naver_all_{f}.pkl")

  # pd.read_pickle('./naver_all.pkl').append(pd.read_pickle('./naver_all2.pkl'), ignore_index= True).to_pickle('./naver_all.pkl')

In [ ]:
%%writefile dc_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('DC갤러리'):
  DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('DC갤러리/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa)
      DC = DC.append(j, ignore_index = True)
      DC.to_pickle(f"./dc_all_{f}.pkl")

In [ ]:
# 백그라운드로 파일 실행
# !nohup /usr/bin/python3 naver_pickle.py > naver.log 2>&1 &
# !nohup /usr/bin/python3 dc_pickle.py > dc.log 2>&1 &

# log 확인
# !tail -f naver.log
# !tail -f dc.log

# 감성사전 만들기

In [2]:
NAVER = pd.read_pickle('./naver_all.pkl')
NAVER

,_id,code,date,title,views,pos,neg,nsmc,shop
0,{'$oid': '61582e2d131fea20b2c41b27'},008770,2021.10.02 18:54,솔직히,5,0,0,0.039333,0.020997
1,{'$oid': '61582e2d131fea20b2c41b28'},008770,2021.10.02 14:44,애 호텔주 아니라니까..,116,0,1,0.065854,0.049799
2,{'$oid': '61582e2d131fea20b2c41b29'},008770,2021.10.02 13:04,솔브레인 화요일 시초에 담아라,204,0,4,0.891313,0.822273
3,{'$oid': '61582e2d131fea20b2c41b2a'},008770,2021.10.02 12:04,기대가 크면 실망이 크다,272,1,1,0.010065,0.013250
4,{'$oid': '61582e2d131fea20b2c41b2b'},008770,2021.10.02 10:20,기가막히게 쌍바닥이네,209,2,0,0.011541,0.032541
...,...,...,...,...,...,...,...,...,...
226144,{'$oid': '61d79b233bec6f3c4b1921a6'},069410,2021.12.30 10:14,10800 물량 뺐다 상가자..,131,0,1,0.004302,0.036052
226145,{'$oid': '61d79b233bec6f3c4b1921a7'},069410,2021.12.30 10:14,10850물량좀 빼라,105,0,0,0.175532,0.021215
226146,{'$oid': '61d79b233bec6f3c4b1921a8'},069410,2021.12.30 09:53,아니 10800물량좀,192,0,0,0.778861,0.036216
226147,{'$oid': '61d79b233bec6f3c4b1921a9'},069410,2021.12.30 09:32,엔털스가 대장되는갑다,241,3,1,0.980688,0.065931


In [3]:
# 중복 제거
NAVER.drop_duplicates(subset = ['code', 'title'], inplace=True, ignore_index= True)
# NAVER[NAVER.duplicated(['code', 'title'])]

In [4]:
NAVER['positive'] = (NAVER['nsmc'] + NAVER.shop) / 2
NAVER

,_id,code,date,title,views,pos,neg,nsmc,shop,positive
0,{'$oid': '61582e2d131fea20b2c41b27'},008770,2021.10.02 18:54,솔직히,5,0,0,0.039333,0.020997,0.030165
1,{'$oid': '61582e2d131fea20b2c41b28'},008770,2021.10.02 14:44,애 호텔주 아니라니까..,116,0,1,0.065854,0.049799,0.057826
2,{'$oid': '61582e2d131fea20b2c41b29'},008770,2021.10.02 13:04,솔브레인 화요일 시초에 담아라,204,0,4,0.891313,0.822273,0.856793
3,{'$oid': '61582e2d131fea20b2c41b2a'},008770,2021.10.02 12:04,기대가 크면 실망이 크다,272,1,1,0.010065,0.013250,0.011657
4,{'$oid': '61582e2d131fea20b2c41b2b'},008770,2021.10.02 10:20,기가막히게 쌍바닥이네,209,2,0,0.011541,0.032541,0.022041
...,...,...,...,...,...,...,...,...,...,...
199508,{'$oid': '61d79b233bec6f3c4b1921a6'},069410,2021.12.30 10:14,10800 물량 뺐다 상가자..,131,0,1,0.004302,0.036052,0.020177
199509,{'$oid': '61d79b233bec6f3c4b1921a7'},069410,2021.12.30 10:14,10850물량좀 빼라,105,0,0,0.175532,0.021215,0.098373
199510,{'$oid': '61d79b233bec6f3c4b1921a8'},069410,2021.12.30 09:53,아니 10800물량좀,192,0,0,0.778861,0.036216,0.407539
199511,{'$oid': '61d79b233bec6f3c4b1921a9'},069410,2021.12.30 09:32,엔털스가 대장되는갑다,241,3,1,0.980688,0.065931,0.523309


In [5]:
from konlpy.tag import Okt, Hannanum, Kkma, Mecab
han = Hannanum()
okt = Okt()
# kkma = Kkma()
mecab = Mecab()


Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
han.tagset

In [ ]:
okt.tagset

In [ ]:
# NAVER['okt'] = NAVER.title.apply(lambda x: okt.pos(x)[0])
# NAVER['kkma'] = NAVER.title.apply(lambda x: kkma.pos(x))

In [6]:
# 전처리
# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
pattern2 = re.compile(r'[^가-힣!]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

NAVER.title = NAVER.title.map(lambda x: pattern3.sub(' ',
                                                           pattern2.sub('',
                                                                        pattern1.sub('', x))))

In [10]:
# 한나눔 N(체언), P(용언)
# NAVER['hannanum'] = NAVER.title.apply(lambda x: [w for w, t in han.pos(x) if t.startswith('N') or t.startswith('P')])

In [ ]:
NAVER

In [ ]:
NAVER.to_pickle('./naver_hannanum.pkl')

mecab 체ㄴ, 용ㅓ, 감탄ㅏ(확ㅣ), (관형ㅏ)


{토큰 : 감ㅓ확률, 토ㄴ: 감ㅓ확률}


# 주가 등락 적용 업데이트
- 10:03 게시글 -> 긍정확률에 수식 적용하려면 % 올랐는지가 있어야 하지 -> 10시 5분에 비해 10:10분 기반으로 올랐는지 내렸는지 학습
- 18:50 게시글 -> 9:00에 비해 9:05가 올랐는지, 09:05 기반으로 학습
- pdf -> 하루종일 영향을 준다. (안 올라오는 경우가 더 많음. 올라올 때만 추가적인 정보로) +a

